**Install libraries**



In [ ]:
!pip install -q prophet chronos-forecasting statsmodels


**Create dummy supply chain data (~6000 rows)**

In [ ]:
import numpy as np
import pandas as pd

# 1. Create date range (~6000 days ≈ 16.4 years)
dates = pd.date_range(start="2010-01-01", periods=6000, freq="D")

n = len(dates)
rng = np.random.default_rng(42)

# 2. Base demand + slight upward trend
base = 100
trend = np.linspace(0, 30, n)  # slowly increasing demand

# 3. Weekly seasonality (higher demand on weekends)
day_of_week = dates.dayofweek  # 0=Mon, 6=Sun
weekly_pattern = np.where(day_of_week >= 5, 20, 0)  # +20 on Sat/Sun

# 4. Promo flag (randomly choose ~10% of days as promotions)
promo_flag = (rng.random(n) < 0.10).astype(int)
promo_effect = promo_flag * 30  # bump when promo is on

# 5. Holiday flag (two fixed days each year: Jan 26, Aug 15)
holiday_dates = []
for year in range(dates.min().year, dates.max().year + 1):
    holiday_dates += [
        f"{year}-01-26",  # Republic Day (example)
        f"{year}-08-15",  # Independence Day (example)
    ]
holidays = pd.to_datetime(holiday_dates)

holiday_flag = dates.isin(holidays).astype(int)
holiday_effect = holiday_flag * 50

# 6. Random noise
noise = rng.normal(0, 10, n)

# 7. Final demand (clip to avoid negative)
demand = base + trend + weekly_pattern + promo_effect + holiday_effect + noise
demand = np.clip(demand, a_min=1, a_max=None)

# 8. Build DataFrame
df = pd.DataFrame({
    "ds": dates,
    "y": demand.round(0).astype(int),
    "promo_flag": promo_flag,
    "holiday_flag": holiday_flag,
})

print("Number of rows:", len(df))
df.head()


**Train–test split + metrics**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1. Set datetime index for easier time-based operations
df_idx = df.set_index("ds").copy()

# 2. Train–test split (80% train, 20% test by time)
split_point = int(len(df_idx) * 0.8)
split_date = df_idx.index[split_point]

train = df_idx.iloc[:split_point].copy()
test = df_idx.iloc[split_point:].copy()

print("Train period:", train.index.min(), "→", train.index.max())
print("Test period :", test.index.min(), "→", test.index.max())
print("Train size:", len(train), "Test size:", len(test))

# 3. Simple metric functions
def rmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(
        np.abs((y_true - y_pred) / np.clip(y_true, 1e-8, None))
    ) * 100

y_test = test["y"].values

# 4. Quick plot to visualize train vs test split
plt.figure(figsize=(12, 4))
plt.plot(train.index, train["y"], label="Train")
plt.plot(test.index, test["y"], label="Test", alpha=0.8)
plt.title("Daily Demand – Train/Test Split")
plt.xlabel("Date")
plt.ylabel("Demand (units)")
plt.legend()
plt.show()


**SARIMAX baseline (statsmodels)**

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# 1. Exogenous variables for train and test
exog_cols = ["promo_flag", "holiday_flag"]
exog_train = train[exog_cols]
exog_test = test[exog_cols]

# 2. Define SARIMAX model
# ARIMA(1,1,1) + seasonal(1,1,1,7) to capture weekly pattern
sarimax_model = SARIMAX(
    endog=train["y"],
    exog=exog_train,
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 7),
    enforce_stationarity=False,
    enforce_invertibility=False,
)

# 3. Fit the model
sarimax_result = sarimax_model.fit(disp=False)

# 4. Forecast over the test horizon
forecast_res = sarimax_result.get_forecast(
    steps=len(test),
    exog=exog_test
)
sarimax_pred = forecast_res.predicted_mean.values

# 5. Evaluate
sarimax_rmse = rmse(y_test, sarimax_pred)
sarimax_mape = mape(y_test, sarimax_pred)

print(f"SARIMAX – RMSE: {sarimax_rmse:.2f}, MAPE: {sarimax_mape:.2f}%")

# 6. Plot
plt.figure(figsize=(12, 4))
plt.plot(train.index, train["y"], label="Train")
plt.plot(test.index, test["y"], label="Test (actual)")
plt.plot(test.index, sarimax_pred, label="SARIMAX forecast")
plt.title("SARIMAX – Supply Chain Demand Forecast")
plt.xlabel("Date")
plt.ylabel("Demand (units)")
plt.legend()
plt.show()


**Prophet model**

In [ ]:
# 0) Clean install: remove any existing Prophet + cmdstanpy
!pip uninstall -y prophet cmdstanpy -q
!pip install -q prophet

from prophet import Prophet
import matplotlib.pyplot as plt

# 1) Prepare train and test for Prophet (needs 'ds' and 'y')
#    (train and test already created in earlier cells)
train_p = train.reset_index()   # columns: ds, y, promo_flag, holiday_flag
test_p = test.reset_index()

# 2) Define Prophet model
m = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
)

# Add regressors for promos and holidays
m.add_regressor("promo_flag")
m.add_regressor("holiday_flag")

# 3) Fit model on train data
m.fit(train_p[["ds", "y", "promo_flag", "holiday_flag"]])

# 4) Build future dataframe for test horizon with same regressors
future = test_p[["ds", "promo_flag", "holiday_flag"]].copy()

# 5) Forecast
forecast = m.predict(future)

# 6) Get predictions and evaluate
prophet_pred = forecast["yhat"].values

prophet_rmse = rmse(y_test, prophet_pred)
prophet_mape = mape(y_test, prophet_pred)

print(f"Prophet – RMSE: {prophet_rmse:.2f}, MAPE: {prophet_mape:.2f}%")

# 7) Plot
plt.figure(figsize=(12, 4))
plt.plot(train.index, train["y"], label="Train")
plt.plot(test.index, test["y"], label="Test (actual)")
plt.plot(test.index, prophet_pred, label="Prophet forecast")
plt.title("Prophet – Supply Chain Demand Forecast")
plt.xlabel("Date")
plt.ylabel("Demand (units)")
plt.legend()
plt.show()

**Chronos-2**

In [ ]:
# 0) Install Chronos + optional pyarrow for pandas
!pip install -q "chronos-forecasting>=2.0" "pandas[pyarrow]"

import pandas as pd
import matplotlib.pyplot as plt
from chronos import Chronos2Pipeline

# 1) Build context (train) and future covariates (test) for Chronos-2
#    Our train/test currently have index=ds and columns: y, promo_flag, holiday_flag

context_df = train.reset_index().rename(columns={"ds": "timestamp", "y": "target"})
context_df["id"] = "sku_A123"
context_df = context_df[["id", "timestamp", "target", "promo_flag", "holiday_flag"]]

future_df = test.reset_index().rename(columns={"ds": "timestamp", "y": "target"})
future_df["id"] = "sku_A123"
# For future_df we drop the target column (we're only providing covariates)
future_cov_df = future_df[["id", "timestamp", "promo_flag", "holiday_flag"]]

print("Context shape:", context_df.shape)
print("Future covariates shape:", future_cov_df.shape)

# 2) Load Chronos-2 pipeline
#    device_map="auto" → uses GPU if available, otherwise CPU.
#    If this is too heavy, you can try a smaller model like "amazon/chronos-bolt-small".
pipeline = Chronos2Pipeline.from_pretrained(
    "amazon/chronos-2",
    device_map="auto",
)

# 3) Forecast with Chronos-2
prediction_length = len(test)

pred_df = pipeline.predict_df(
    context_df,
    future_df=future_cov_df,
    prediction_length=prediction_length,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column="id",
    timestamp_column="timestamp",
    target="target",
)

# 4) Extract predictions for our single id
pred_df_single = pred_df[pred_df["id"] == "sku_A123"].sort_values("timestamp")

chronos_pred = pred_df_single["predictions"].values

# 5) Evaluate
chronos_rmse = rmse(y_test, chronos_pred)
chronos_mape = mape(y_test, chronos_pred)

print(f"Chronos-2 – RMSE: {chronos_rmse:.2f}, MAPE: {chronos_mape:.2f}%")

# 6) Plot
plt.figure(figsize=(12, 4))
plt.plot(train.index, train["y"], label="Train")
plt.plot(test.index, test["y"], label="Test (actual)")
plt.plot(test.index, chronos_pred, label="Chronos-2 forecast")
plt.title("Chronos-2 – Supply Chain Demand Forecast")
plt.xlabel("Date")
plt.ylabel("Demand (units)")
plt.legend()
plt.show()


**Final metrics table**

In [ ]:
# Final comparison table
results = pd.DataFrame({
    "Model": [
        "SARIMAX (statsmodels)",
        "Prophet",
        "Chronos-2",
    ],
    "RMSE": [
        sarimax_rmse,
        prophet_rmse,
        chronos_rmse,
    ],
    "MAPE (%)": [
        sarimax_mape,
        prophet_mape,
        chronos_mape,
    ],
})

results.sort_values("MAPE (%)").reset_index(drop=True)
